In [5]:
import unidecode
import string
import random
import re
import numpy as np

file = open('./data/log.txt').read()
file_len = len(file)
print('file_len =', file_len)
all_characters = "".join(list(set([x for x in file])))
all_chars = list(set([x for x in file]))
printable = list(string.printable)
all_characters = "".join(list(set(printable + all_chars)))
n_characters = len(all_characters)
print(all_characters)
print(len(all_characters))

file_len = 502890
2an&<t@T_"sKW;:0mopGY1?9S(D cu=5%x!6U+l,HN/'y…{Q]‘CfR7E”[~\dkgB*j
LJ	Ih`-^3wXv|8﻿>Zb’PO“#qiA}←$⅓M4e.
109


In [6]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import autograd

class RNN(nn.Module):
    def __init__(self, *, input_size, embedding_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        # map inputs to embeddings
        self.embedding_layer = nn.Embedding(input_size, embedding_size)
        # forward embeddings through LSTM
        self.LSTM = nn.LSTM(embedding_size, hidden_size, n_layers)
        # compute a linear transformation to output space
        self.linear = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        tmp = input
        input = self.embedding_layer(input.view(1, -1))
        output, hidden = self.LSTM(input.view(tmp.shape[0], 1, -1), hidden)
        output = self.linear(output.view(tmp.shape[0], -1))
        scores = F.softmax(output)
        return output, hidden, scores

    def init_hidden(self):
        # initialize hidden state h_0 and cell state c_0
        return (autograd.Variable(torch.zeros(self.n_layers, 1, self.hidden_size)),
                autograd.Variable(torch.zeros(self.n_layers, 1, self.hidden_size)))

In [7]:
seq_len = 200

def get_seq(seq_len = 200):
    start = np.random.randint(0, file_len - seq_len)
    seq = file[start:start + seq_len]
    return seq

def to_vector(seq, chars = all_characters):
    unknown_idx = 0
    vec = []
    for s in seq:
        try:
            vec.append(chars.index(s))
        except ValueError:
            print('{} not found'.format(s))
            vec.append(unknown_idx)
    return autograd.Variable(torch.LongTensor(vec))

def generate_training_set():
    seq = get_seq()
    inputs, labels = to_vector(seq[:-1]), to_vector(seq[1:])
    return inputs, labels



In [30]:
def get_most_likely(output, temperature = 0.8):
    # Convert output to a multinomial and sample the most likely element
    output_dist = output.data.view(-1).div(temperature).exp()
    top_1_idx = torch.multinomial(output_dist, 1)[0]
    # index into the list of all characters, and use this predicted character as the next input to the RNN
    # Add predicted character to string and use as next input
    predicted_char = all_characters[top_1_idx]
    return top_1_idx

def evaluate(start_char='A', predict_len=100, temperature=0.8):
    # initialize the hidden state
    hidden = rnn.init_hidden()
    # convert the starting string to vector
    start_vector = to_vector(start_char)
    predicted = start_char
    input = start_vector[-1]
    for p in range(predict_len):
        # feed the input to the network
        output, hidden, scores = rnn(input, hidden)
        # Convert output to a multinomial and sample the most likely element
        output_dist = output.data.view(-1).div(temperature).exp()
        top_1_idx = torch.multinomial(output_dist, 1)[0]
        # index into the list of all characters, and use this predicted character as the next input to the RNN
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_1_idx]
        predicted += predicted_char
        input = to_vector(predicted_char)

    return predicted

In [ ]:
n_epochs = 5000
hidden_size = 200
embedding_size = 100
n_layers = 4
        
rnn = RNN(input_size = len(all_characters),
          embedding_size = 100,
          hidden_size = 200, 
          output_size = len(all_characters),
          n_layers = 4)

optim = torch.optim.Adam(rnn.parameters(), lr = 0.005)
criterion = nn.CrossEntropyLoss()
all_losses = []
teacher_forcing_prob = 0.5
            
for epoch in range(n_epochs):
    # re-init hidden and zero the grads
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    inputs, labels = generate_training_set()
    # one by one inputs
    loss = 0
    next_generated = None
    for i in range(len(inputs)):
        input, label = inputs[i] if i == 0 or np.random.random() < teacher_forcing_prob else next_generated, labels[i]
        output, hidden, scores = rnn(input, hidden)
        loss+=criterion(output, label)
        idx = get_most_likely(output)
        next_generated = autograd.Variable(torch.LongTensor([idx]))
    loss.backward(retain_graph = True)
    optim.step()
    if epoch % 25 == 0:
        print('EPOCH: {}'.format(epoch))
        print('loss: {}'.format(loss.data[0]/len(inputs)))
        predicted = evaluate()
        print('predicted: {}'.format(predicted))
        
        
    # forward ths input sequence through the RNN
#     outputs, hidden, scores = rnn(inputs, hidden)
#     loss = criterion(outputs, labels)
#     loss.backward(retain_graph = True)
#     all_losses.append(loss.data[0])
#     optim.step()
#     if epoch % 25 == 0:
#         print('EPOCH: {}'.format(epoch))
#         predicted = evaluate()
#         print('predicted: {}'.format(predicted))
#         print('LOSS: {}'.format(loss.data[0]))


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


EPOCH: 0
loss: 4.703518201358354
predicted: A.N{$u ’] Er'o
>UW`+KYI*`	piK+rR’$’
+U+\XNta>⅓,r!zQiz7T_/";uy[L|H;1;oNd\C5
soh→H’ASXyNvq﻿eL
u~95uU
EPOCH: 25
loss: 3.190554211487123
predicted: Aj.tiol e  o eye eg k/fr  ya cak      eoys aognenFrrx  ieedylis o svoh  l   ritv ndeita itltldetoueuu
EPOCH: 50
loss: 3.1005488256713254
predicted: A
u sr edoeobtteotdeea vna  toa  e dr   nd I h t n  o  riImvfs
annhatr     t  i a  ae  td deeagy   et
EPOCH: 75
loss: 3.0825851670461684
predicted: Atttnhoeut ertpege l  nnt    nnik at nteec ny* ie sm    n a  eu csio u   wPee rt rdso onf tmLteomo en
EPOCH: 100
loss: 3.1522456030150754
predicted: Aqtseofl ld po  rgiteui ahrtaa a i ltk  dtm mItk sn n yatoafke  taue in s
 l i un dtct   v eaun s * e
EPOCH: 125
loss: 3.367339627826633
predicted: Alt  mlt k  onyu neosb  Uf  a cosu uet  d ai s oere n e 
 et iytmslhe dm I    htifcty ungnhoo tsskM u
EPOCH: 150
loss: 3.226645924937186
predicted: ANa,lfi nIseo o  ramar  ld btigy sno,yohyd asmor ad ri on B sehcko n

In [ ]:
# sample the trained network
evaluate()

In [ ]:
evaluate()

In [ ]:
for i in range(100):
    print(evaluate(predict_len=200))
    print('----')

In [ ]:
chars = 'ABCDEFGHIJKLMOPQRSTUVWXYZ'
for i in range(100):
    print(evaluate(start_char = chars[np.random.randint(0, len(chars)-1)], predict_len = 200))
    print('----')